In [ ]:

#  1.Data Cleaning and Finding the median of the Male employees incomes and Female employees income

import datatable as dt
import datascience as ds
import pandas as pd
import numpy as np
data = pd.read_csv("Desktop\\multipleChoiceResponses.csv")
data = data.reindex(data.index.drop(0)).reset_index(drop=True)
data = data.dropna(how='any', subset=['Q9'])
mean_dict = {'0-10,000': 5000, '10-20,000': 15000, '20-30,000': 25000, '30-40,000': 35000,
       '40-50,000': 45000, '50-60,000': 55000, '60-70,000': 65000, '70-80,000': 75000,
       '80-90,000': 85000, '90-100,000': 95000, '100-125,000': 112500,
       '125-150,000': 137500, '150-200,000': 175000, '200-250,000': 225000,
       '250-300,000': 275000, '300-400,000': 350000, '400-500,000': 450000,
       '500,000+':500000}
data['Mean Salary'] = data['Q9'].apply(lambda value: mean_dict[value])
input_table = ds.Table.from_df(data, keep_index=False)
income_table = input_table.select(['Q1', 'Mean Salary'])
income_table = income_table.relabeled('Q1', 'Gender')
male_and_female = income_table.where('Gender', ds.are.containing('ale'))
male_and_female_medians = male_and_female.group('Gender', np.median)
male_and_female_medians.show()


In [ ]:

#2.Ploting the overlaid Histogram graph for the income of male and female employees

import matplotlib.pyplot as plt
%matplotlib inline
male_female_income_df = male_and_female.to_df()
plt.hist(male_female_income_df[male_female_income_df['Gender']=='Male']['Mean Salary'], bins=20, alpha=0.5, label='Male Income')
plt.hist(male_female_income_df[male_female_income_df['Gender']=='Female']['Mean Salary'], bins=20, alpha=0.5, label='Female Income')
plt.legend(loc='upper center')
plt.title('Overlaid Histogram of Male and Female Incomes')
plt.show()

In [ ]:

# 3.1.Selecting the sample which includes 500 employees

male_income = male_and_female.where('Gender', ds.are.equal_to('Male'))
female_income = male_and_female.where('Gender', ds.are.equal_to('Female'))
male_sample = male_income.sample(250)
female_sample = female_income.sample(250)
male_sample_df = male_sample.to_df()
female_sample_df = female_sample.to_df()
sample = female_sample_df.append(male_sample_df, ignore_index=True)
sample_table = ds.Table.from_df(sample, keep_index=False)

 
#3.3.Drawing the sample income histogram and, Plotting the medians of male and female incomes population

plt.hist(male_female_income_df['Mean Salary'], bins = 20, alpha=0.5, label='Income')
plt.scatter(25000, 0, color='red', s=30)
plt.scatter(35000, 0, color='yellow', s=30)

In [ ]:

#3.4. Drawing the histogram of test statistics of sample and plotting the observed statistcs on the graph using Testing Hypothesis
result = ds.make_array()
for i in np.arange(500):
    male_sample = male_income.sample(250)
    female_sample = female_income.sample(250)
    male_sample_df = male_sample.to_df()
    female_sample_df = female_sample.to_df()
    sample = female_sample_df.append(male_sample_df, ignore_index=True)
    sample_table = ds.Table.from_df(sample, keep_index=False)
    male_and_female_means = sample_table.group('Gender', np.average)
    male_and_female_means_df = male_and_female_means.to_df()
    result = np.append(result, male_and_female_means_df.iloc[1]['Mean Salary average'] - male_and_female_means_df.iloc[0]['Mean Salary average'])
population_mean = male_and_female.group('Gender', np.average).to_df()
observed_statistic = population_mean.iloc[1]['Mean Salary average'] - population_mean.iloc[0]['Mean Salary average']
plt.hist(result, bins = 20, alpha=0.5, label='Income')
plt.scatter(observed_statistic, 0, color='red', s=30)

In [ ]:

# 3.6.Drawing the histogram of incomes of bootstrap sample using Bootstrap

bootstrap_result = ds.make_array()
for i in np.arange(500):
    bootstrap_sample = sample_table.sample()
    bootstrap_male_and_female_means = bootstrap_sample.group('Gender', np.average)
    bootstrap_male_and_female_means_df = bootstrap_male_and_female_means.to_df()
    bootstrap_result = np.append(bootstrap_result, (bootstrap_male_and_female_means_df.iloc[1]['Mean Salary average'] - bootstrap_male_and_female_means_df.iloc[0]['Mean Salary average']))
plt.hist(bootstrap_result, bins = 20, alpha=0.5, label='Income')

In [ ]:

#3.7. Finding p-value using the A/B Testing

shuffled_results = ds.make_array()
for i in np.arange(500):
    shuffled_Genders = male_and_female.sample(with_replacement = False).column(0)
    original_and_shuffled = male_and_female.with_column('Shuffled Gender', shuffled_Genders)
    shuffled_only = original_and_shuffled.drop('Gender')
    shuffled_group_means = shuffled_only.group('Shuffled Gender', np.average)
    shuffled_group_means_df = shuffled_group_means.to_df()
    shuffled_results = np.append(shuffled_results, (shuffled_group_means_df.iloc[1]['Mean Salary average'] - shuffled_group_means_df.iloc[0]['Mean Salary average']))
plt.hist(shuffled_results, bins = 20, alpha=0.5, label='Income')

p_value = np.count_nonzero(shuffled_results <= observed_statistic) / i
p_value

In [ ]:
pip install datatable